<a href="https://colab.research.google.com/github/Friedrichz/crypto_tooling/blob/main/track_vc_tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install messari

In [117]:
from messari.messari import Messari
import pandas as pd
import numpy as np

messari_api_key = "88bbfa31-1b0c-442b-b748-cc13ed2cab1a"
messari = Messari(messari_api_key)

In [4]:
# Fetch raw data
li_dic = [] 
for i in range(1, 5, 1):
    data = messari.get_all_assets(page=i, limit=500, to_dataframe=True)
    li_dic.append(data)

# Merge raw API data into one list with records 
data = [j for i in li_dic for j in i["data"]]
len(data)

# Cleanup records 

In [108]:
def get_nested_items(list_records, attr): 
  if list_records is None:
    return []
  else: 
    return [i[attr] for i in list_records]

def clean_sales_round_title(title): 
  if title is None: pass
  else:
    # Remove digits
    title = ''.join([i for i in title if not i.isdigit()]) 
    # Lowercase & strip trailing/leading spaces
    title = title.lower().strip()
    return title

def clean_sales_rounds(list_records): 
  if list_records is None:
    pass
  else:
    [i.update(title_clean=clean_sales_round_title(i["title"])) for i in list_records]
    return

# Get list of sales_rounds labels to identify venture round / seed sale 
sales_rounds_titles = set(j for i in data for j in get_nested_items(i["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"], "title"))

# Taken manually from list of titles in data 
private_sales_rounds_titles = [
  'Fundraising ', 
  'Pre Seed Round', 
  'Pre-Sale SAFT', 
  'Private Presale', 
  'Private Sale', 
  'Private Sale1 ',
  'SAFT',
  'Seed', 
  'Seed ', 
  'Seed Funding', 
  'Seed Round', 
  'Seed Round (SAFT) ', 
  'Seed Sale', 
  'Seed Round (Crypto)', 
  'Seed Token Sale',
  'Strategic Funding Round',
  'Strategic Investments',
  'Strategic Private Sale',
  'Strategic Round',
  'Strategic Sale',
  'Token Pre-Sale',
  'Token Private Round',
  'Token Seed Round',
  'Venture Capital',
  'Venture Funding',
  'Venture Round',
]

# Clean up sales round titles & add "title_clean" to records
private_sales_rounds_titles_clean = set(clean_sales_round_title(i) for i in private_sales_rounds_titles)
print(len(private_sales_rounds_titles_clean)) # Include 24 titles that point to private token sales
_ = [clean_sales_rounds(j["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"]) for j in data]

24


In [111]:
# Extract (current) market data --> metrics: token_sale_stats>sale_proceeds_usd, supply_distribution
def get_market_data(record): 
  # Get current supply (see docs: https://messari.io/article/messari-proprietary-methods)
  circ_supply = record["metrics"]["supply"]["circulating"] # liquid supply that excludes project, foundation or founder units which have not been yet sold
  liquid_supply = record["metrics"]["supply"]["liquid"] # supply taht is visible on-chain and which is not known to have any programmatic or contractual restrictions

  # Get ath price data
  ath_price = record["metrics"]["all_time_high"]["price"]
  ath_date = record["metrics"]["all_time_high"]["at"]

  # Get cycle low price data
  cl_price = record["metrics"]["cycle_low"]["price"]
  cl_date = record["metrics"]["cycle_low"]["at"]

  # Get current mcap
  mcap = record["metrics"]["marketcap"]["current_marketcap_usd"]
  liquid_mcap = record["metrics"]["marketcap"]["liquid_marketcap_usd"]

  # Get current price data
  current_price = record["metrics"]["market_data"]["price_usd"]

  # Get ROI data
  pct_changes_keys = ["percent_change_last_1_year", "percent_change_year_to_date", "percent_change_last_3_months", "percent_change_last_1_month", "percent_change_month_to_date", "percent_change_last_1_week",]
  pct_changes = {k:v for k,v in record["metrics"]["roi_data"].items() if k in pct_changes_keys}

  return dict(
      current_price = current_price,
      circ_supply = circ_supply, 
      liquid_supply = liquid_supply, 
      ath_price = ath_price, 
      ath_date = ath_date, 
      cl_price = cl_price, 
      cl_date = cl_date, 
      mcap = mcap, 
      liquid_mcap = liquid_mcap,
      **pct_changes       
  )


In [119]:
# Extract initial private fundraising data -->  profile: investors>organizations, economics>launch>fundraising
def get_fundraise_data(record):
  # Init
  investors = []
  first_private_sale_date = np.nan
  last_private_sale_date = np.nan
  num_tokens_private_sale = np.nan
  tot_usd_raised_private_sale = np.nan
  min_price_private_sale = np.nan
  max_price_private_sale = np.nan
  wavg_price_private_sale = np.nan 
  init_supply = np.nan
  fdv_private_sale = np.nan
  asset_collected = np.nan

  # Get list of private investors
  investors = [i["name"] for i in record["profile"]["investors"]["organizations"]]

  # Filter records of private token sales
  private_rounds = [i for i in record["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"] if i["title_clean"] in private_sales_rounds_titles_clean]
  if private_rounds:
    # Get earliest & last date of private sales
    first_private_sale_date = min([i["end_date"] for i in private_rounds])
    last_private_sale_date = max([i["end_date"] for i in private_rounds])

    # Get total number of tokens sold during private sales
    num_tokens_private_sale = sum([i["native_tokens_allocated"] for i in private_rounds])

    # Get total USD amount raised in private sales
    tot_usd_raised_private_sale = sum([i["amount_collected_in_usd"] for i in private_rounds])

    # Get last private sale token price 
    min_price_private_sale = min([i["equivalent_price_per_token_in_usd"] for i in private_rounds])
    max_price_private_sale = max([i["equivalent_price_per_token_in_usd"] for i in private_rounds])

    # Get weighted avg token price of private sales
    wavg_price_private_sale = sum([i["amount_collected_in_usd"]*i["equivalent_price_per_token_in_usd"] for i in private_rounds]) / tot_usd_raised_private_sale

    # Get total token supply
    init_supply = record["profile"]["economics"]["launch"]["initial_distribution"]["initial_supply"]

    # Calc FDV based on last private sale token price
    fdv_private_sale = max_price_private_sale * init_supply

    # Safety check: currency collected in private fundraise
    asset_collected = set(i["asset_collected"] for i in private_rounds)

  return dict(
        investors = investors, 
        first_private_sale_date = first_private_sale_date, 
        last_private_sale_date = last_private_sale_date, 
        num_tokens_private_sale = num_tokens_private_sale, 
        tot_usd_raised_private_sale = tot_usd_raised_private_sale, 
        min_price_private_sale = min_price_private_sale, 
        max_price_private_sale = max_price_private_sale, 
        wavg_price_private_sale = wavg_price_private_sale, 
        init_supply = init_supply, 
        fdv_private_sale = fdv_private_sale,
        asset_collected = asset_collected,
    )

In [120]:
# Get final data for all records
# record = data[13]

final_records = []

for record in data: 
  final_records.append(
      dict(
        name = record["name"], 
        link = "https://messari.io/asset/{}".format(record["slug"]),
        ** get_market_data(record),
        ** get_fundraise_data(record), 
      )
  )

TypeError: ignored

In [ ]:
data[24]["name"]
# [i["name"] for i in data[24]["profile"]["investors"]["organizations"]]
[i for i in data[24]["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"]]

In [ ]:
url = "https://data.messari.io/api/v2/assets?with-profiles"
page = 2
assets_bytes = urllib.request.urlopen(url + "?page={}".format(str(page))).read()
assets_dict = json.loads(assets_bytes.decode('utf-8'))

In [ ]:
len(assets_dict)
len(assets_dict["data"])

2